In [ ]:
import cv2 as cv
import numpy as np
import os

def remove_gray_background(image, brightness, contrast):
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)

    hsv[:, :, 2] = np.clip(hsv[:, :, 2] * contrast + brightness, 0, 255)

    adjusted_bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)

    lower_gray = np.array([0, 0, 100], dtype=np.uint8)
    upper_gray = np.array([255, 5, 255], dtype=np.uint8)
    gray_mask = cv.inRange(hsv, lower_gray, upper_gray)

    non_gray_mask = cv.bitwise_not(gray_mask)

    image_without_gray = cv.bitwise_and(adjusted_bgr, adjusted_bgr, mask=non_gray_mask)

    return image_without_gray

def extract_white_blood_cells(img):
    # Convert back to BGR
    img = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    adjusted_bgr = cv.cvtColor(img, cv.COLOR_HSV2BGR)

    lower_purple = np.array([130, 50, 50])
    upper_purple = np.array([170, 255, 255])

    purple_mask = cv.inRange(img, lower_purple, upper_purple)

    result = cv.bitwise_and(adjusted_bgr, adjusted_bgr, mask=purple_mask)
    return result

def remove_small(img, min_area_threshold=90):

    lower_purple = np.array([130, 50, 50])
    upper_purple = np.array([170, 255, 255])

    purple_mask = cv.inRange(img, lower_purple, upper_purple)

    contours, _ = cv.findContours(purple_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    filtered_mask = np.zeros_like(purple_mask)
    for contour in contours:
        if cv.contourArea(contour) > min_area_threshold:
            cv.drawContours(filtered_mask, [contour], 0, 255, thickness=cv.FILLED)

    result = cv.bitwise_and(img, img, mask=filtered_mask)
    result = cv.cvtColor(result, cv.COLOR_BGR2GRAY)

    _, binary_image = cv.threshold(result, 10, 255, cv.THRESH_BINARY)
    return binary_image

def calculate_iou(pred_mask, true_mask):
    intersection = np.logical_and(pred_mask, true_mask)
    union = np.logical_or(pred_mask, true_mask)
    iou = np.sum(intersection) / np.sum(union)
    return iou

def preprocess_ground_truth(true_mask):
    _, binary_true_mask = cv.threshold(true_mask, 10, 255, cv.THRESH_BINARY)
    return binary_true_mask

string_paths = ["baso", "eosi", "lymp", "mixt", "mono", "neut"]
images = [53, 39, 52, 8, 48, 50]


def main():
    output_folder = "saved"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i in range(len(string_paths)):
        folder_path = os.path.join("output_pre", string_paths[i])

        for j in range(1, images[i] + 1):
            path = os.path.join(folder_path, f"{j}.bmp")

            if not os.path.isfile(path):
                print(f"Error: File not found - {path}")
                continue

            noise_image = cv.imread(path)

            result_image = remove_gray_background(noise_image, brightness=1.5, contrast=4)
            median_blur = cv.medianBlur(result_image, 3)
            result = extract_white_blood_cells(result_image)
            result = cv.medianBlur(result, 3)
            result = remove_small(result)

            ground_truth_path = os.path.join("output", string_paths[i], f"{j}.bmp")
            ground_truth_mask = cv.imread(ground_truth_path, cv.IMREAD_GRAYSCALE)
            preprocessed_ground_truth = preprocess_ground_truth(ground_truth_mask)

            iou = calculate_iou(result, preprocessed_ground_truth)
            print(f"IoU (Jaccard index) : {iou}")

            if iou > 0.0:
                output_path = os.path.join(output_folder, f"{string_paths[i]}_{j}_IoU_{iou:.4f}.bmp")
                cv.imwrite(output_path, result)

    print("Result images saved in the 'saved' folder.")

main()
